In [2]:
import numpy   as np
import pandas  as pd

In [3]:
brats = pd.read_csv('/Users/suzukishinji/kaggle/christmas/child_wishlist_v2.csv', header=None, index_col=0).as_matrix()
gifts = pd.read_csv('/Users/suzukishinji/kaggle/christmas/gift_goodkids_v2.csv', header=None, index_col=0).as_matrix()
submission = pd.read_csv('/Users/suzukishinji/kaggle/christmas/sample_submission_random_v2.csv')

In [14]:
submission.head()

,ChildId,GiftId
0,0,61
1,1,61
2,2,61
3,3,322
4,4,322


In [51]:
display(child_wish.shape)
display(gift.shape)

(1000000, 100)

(1000, 1000)

In [4]:
TRIPS_COUNT =  5_001
TWINS_COUNT = 40_000
TWINS_START = TRIPS_COUNT
TWINS_END   = TWINS_START + TWINS_COUNT
GIFTS_LIMIT =  1_000

BRAT_PREF_COUNT = brats.shape[1]
GIFT_PREF_COUNT = gifts.shape[1]

In [5]:
assigned = [None] * len(brats)
gift_cnt = [0]    * len(gifts)

def assign(brat_id, gift_id):
    assigned[brat_id]  = gift_id
    gift_cnt[gift_id] += 1
    
def find_best_preferred(brat_id, copies=0):
    for preference_order in range(BRAT_PREF_COUNT):
        preference = brats[brat_id, preference_order]
        if gift_cnt[preference] + copies < GIFTS_LIMIT:
            return preference
    return np.argmin(gift_cnt) # least_popular_gift

def display_progress(curr_id, total=len(brats)):
    completed = curr_id + 1
    percent   = completed / total * 100
    if percent % 10 == 0:
        percent_str = str(percent).rjust(5)
        print(f"Completed: {percent_str}%")

In [6]:
for trip_1 in range(0, TRIPS_COUNT, 3):
    trip_1_preference = find_best_preferred(trip_1, 2)
    assign(trip_1,     trip_1_preference)
    assign(trip_1 + 1, trip_1_preference)
    assign(trip_1 + 2, trip_1_preference)

In [7]:
for twin_1 in range(TWINS_START, TWINS_END, 2):
    twin_1_preference = find_best_preferred(twin_1, 1)
    assign(twin_1,     twin_1_preference)
    assign(twin_1 + 1, twin_1_preference)

In [8]:
for brat_id in range(TWINS_END, len(brats)):
    best_gift = find_best_preferred(brat_id)
    assign(brat_id, best_gift)
    
    display_progress(brat_id)

Completed:  10.0%
Completed:  20.0%
Completed:  30.0%
Completed:  40.0%
Completed:  50.0%
Completed:  60.0%
Completed:  70.0%
Completed:  80.0%
Completed:  90.0%
Completed: 100.0%


In [9]:
print("Quantity of Gifts:")
pd.Series(gift_cnt).value_counts()

Quantity of Gifts:


1000    1000
dtype: int64

In [11]:
submission = pd.DataFrame({
    "ChildId": range(len(brats)),
    "GiftId":  assigned
})

In [12]:
submission.to_csv("submit_1225_1.csv", index=False)

In [13]:
submit_1 = pd.read_csv('/Users/suzukishinji/kaggle/recluit/submit_1225_1.csv')
submit_1.head()

,id,visitors
0,air_00a91d42b08b08d9_2017-04-23,17.912939
1,air_00a91d42b08b08d9_2017-04-24,16.944117
2,air_00a91d42b08b08d9_2017-04-25,19.992022
3,air_00a91d42b08b08d9_2017-04-26,23.235420
4,air_00a91d42b08b08d9_2017-04-27,23.813169
